In [2]:
# Merge the two dataets
import pandas as pd

In [3]:
base_dir = "/home/dermot.kelly/Dermot_analysis/Phd/Paper_2/microbiome_ml/"

# abundance table
abund = pd.read_csv(f"{base_dir}/data/genus_relative_abundance.csv", sep=",", header=0, dtype={'Unnamed: 0': str})
# rename first column
abund = abund.rename(columns={'Unnamed: 0': 'sample_id'})

growing = pd.read_csv(f"{base_dir}/data/full_growings.csv", sep=",", header=0, low_memory=False)

growing.head()

,ANI_ID,ch4_g_day2_1v3,tag,source,date,animal_type,diet_type,hr_off_feed,lot_no,chamber,...,methane_per_rumen,methane_per_ct_fat_kg,DTS,Residuals_ADG_growings,Residuals_MBW_growings,Residuals_CW_growings,Residuals_DMI_growings,Residuals_MBW_ADG_growings,Residuals_LW_growings,Residuals_MBW_DMI_growings
0,1005808,13.572887,372043005104702,BETT,2022-07-28,Lamb,Grass,5,5,10,...,NaN,NaN,NaN,NaN,0.864269,NaN,NaN,NaN,0.860072,NaN
1,1005836,10.289375,372043005104714,BETT,2022-07-28,Lamb,Grass,6,6,4,...,NaN,NaN,NaN,-0.636192,-1.278107,NaN,NaN,-1.642544,-1.241643,NaN
2,1005838,12.275618,372043005104716,BETT,2022-07-28,Lamb,Grass,6,6,10,...,NaN,NaN,NaN,0.596136,0.739667,NaN,NaN,0.619110,0.761671,NaN
3,1005845,14.158780,372043005104717,BETT,2022-07-28,Lamb,Grass,5,5,9,...,NaN,NaN,NaN,1.606109,0.712896,NaN,NaN,0.752933,0.735138,NaN
4,1005849,8.965034,372043005104725,BETT,2022-07-28,Lamb,Grass,6,6,11,...,NaN,NaN,NaN,-1.400531,-1.180905,NaN,NaN,-1.214046,-1.172950,NaN


In [5]:
abund['sample_id'] = abund['sample_id'].astype(str)   # make sure it's string

# always treat tag as string first
growing["key7"] = growing["tag"].astype(str).str[-7:].str.zfill(7)
growing["key5"] = growing["tag"].astype(str).str[-5:].str.zfill(5)

growing.head()

,ANI_ID,ch4_g_day2_1v3,tag,source,date,animal_type,diet_type,hr_off_feed,lot_no,chamber,...,DTS,Residuals_ADG_growings,Residuals_MBW_growings,Residuals_CW_growings,Residuals_DMI_growings,Residuals_MBW_ADG_growings,Residuals_LW_growings,Residuals_MBW_DMI_growings,key7,key5
0,1005808,13.572887,372043005104702,BETT,2022-07-28,Lamb,Grass,5,5,10,...,NaN,NaN,0.864269,NaN,NaN,NaN,0.860072,NaN,5104702,04702
1,1005836,10.289375,372043005104714,BETT,2022-07-28,Lamb,Grass,6,6,4,...,NaN,-0.636192,-1.278107,NaN,NaN,-1.642544,-1.241643,NaN,5104714,04714
2,1005838,12.275618,372043005104716,BETT,2022-07-28,Lamb,Grass,6,6,10,...,NaN,0.596136,0.739667,NaN,NaN,0.619110,0.761671,NaN,5104716,04716
3,1005845,14.158780,372043005104717,BETT,2022-07-28,Lamb,Grass,5,5,9,...,NaN,1.606109,0.712896,NaN,NaN,0.752933,0.735138,NaN,5104717,04717
4,1005849,8.965034,372043005104725,BETT,2022-07-28,Lamb,Grass,6,6,11,...,NaN,-1.400531,-1.180905,NaN,NaN,-1.214046,-1.172950,NaN,5104725,04725


In [6]:
# --- Pass 1: try to match on 7-digit key --------------------------
m7 = abund.merge(
        growing,
        left_on="sample_id",
        right_on="key7",
        how="left",
        suffixes=("", "_meta")
     )

# rows where 7-digit **didn’t** match
unmatched = m7[m7["key7"].isna()].copy()

# --- Pass 2: match those leftovers on 5-digit key -----------------
if not unmatched.empty:
    m5 = unmatched.drop(columns=growing.columns, errors="ignore").merge(
            growing,
            left_on="sample_id",
            right_on="key5",
            how="left",
            suffixes=("", "_meta")
        )

    # overwrite the unmatched slice in m7 with the m5 results
    m7.update(m5)

merged = m7


In [10]:
# Save as csv
output_path = F"{base_dir}/data/growings_microbiome.csv"

merged.to_csv(output_path)